In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input, Conv1D, LeakyReLU,BatchNormalization,MaxPooling1D, Flatten, Dropout, Bidirectional, LSTM, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.constraints import max_norm

In [113]:
from sklearn.metrics import mean_absolute_error


In [2]:
abp_data = pd.read_csv("D:\\Internship\\Blood Pressure\\ABPn.csv")
ppg_data = pd.read_csv("D:\\Internship\\Blood Pressure\\PPGn.csv")
ecg_data = pd.read_csv("D:\\Internship\\Blood Pressure\\ECGn.csv")
sbp_data = pd.read_csv("D:\\Internship\\Blood Pressure\\SBPn.csv")
dbp_data = pd.read_csv("D:\\Internship\\Blood Pressure\\DBPn.csv")
# Standardize the ECG and PPG data
scaler = StandardScaler()
ppg_data = scaler.fit_transform(ppg_data)
ecg_data = scaler.fit_transform(ecg_data)

# Combine SBP and DBP into a single output array
bp_data = np.hstack((sbp_data, dbp_data))

# Reshape the data to add a feature dimension
ppg_data = np.expand_dims(ppg_data, axis=-1)
ecg_data = np.expand_dims(ecg_data, axis=-1)

# Split the data into training and testing sets
ppg_train, ppg_test, ecg_train, ecg_test, bp_train, bp_test = train_test_split(
    ppg_data, ecg_data, bp_data, test_size=0.2, random_state=42
)

In [102]:
for i in range(len(bp_test)):
    print(f"bp[{i}] = {bp_test[i]}")

bp[0] = [148.6311269   60.43028711]
bp[1] = [120.8605742   52.60759622]
bp[2] = [141.1594613   70.09128963]
bp[3] = [151.5632556   70.38435426]
bp[4] = [123.0118142  56.3233744]
bp[5] = [125.3586215   54.36770167]
bp[6] = [120.2738724   53.78099986]
bp[7] = [133.5724469   54.95440349]
bp[8] = [116.0535918   64.66959406]
bp[9] = [120.665007   52.8031635]
bp[10] = [115.9713924   51.43419259]
bp[11] = [129.2699669   55.34553803]
bp[12] = [111.4733452   51.43419259]
bp[13] = [123.598516   54.1721344]
bp[14] = [119.8827379   54.56326895]
bp[15] = [132.2209903   67.35601979]
bp[16] = [119.1004688   53.97656713]
bp[17] = [134.354716    57.10564348]
bp[18] = [140.9152408  63.9857766]
bp[19] = [142.2340316   70.28666605]
bp[20] = [128.2921306   55.73667258]
bp[21] = [120.8891581   63.10658272]
bp[22] = [121.8384106   54.95440349]
bp[23] = [125.9453233   55.54110531]
bp[24] = [122.0339779   55.14997076]
bp[25] = [131.4883287   69.35862807]
bp[26] = [135.7377658   63.83924428]
bp[27] = [149.26758

In [3]:
m1 = load_model("mdl_16_32_leakyrelu_5_5.keras")

In [4]:
m1.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ppg_input           │ (None, 1024, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 1024, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 1022, 16)  │         64 │ ppg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 1022, 16)  │         64 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_8       │ (None, 1022, 16)  │          0 │ conv1d_8[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_9       │ (None, 1022, 16)  │          0 │ conv1d_9[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_8     │ (None, 511, 16)   │          0 │ leaky_re_lu_8[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_9     │ (None, 511, 16)   │          0 │ leaky_re_lu_9[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 509, 32)   │      1,568 │ max_pooling1d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 509, 32)   │      1,568 │ max_pooling1d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_10      │ (None, 509, 32)   │          0 │ conv1d_10[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_11      │ (None, 509, 32)   │          0 │ conv1d_11[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_10    │ (None, 254, 32)   │          0 │ leaky_re_lu_10[0… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_11    │ (None, 254, 32)   │          0 │ leaky_re_lu_11[0… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_4     │ (None, 64)        │     16,640 │ max_pooling1d_10… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 64)        │     16,640 │ max_pooling1d_11… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 128)       │          0 │ bidirectional_4[… │
│ (Concatenate)       │                   │            │ bidirectional_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │      4,128 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 122,216 (477.41 KB)

 Trainable params: 40,738 (159.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 81,478 (318.28 KB)

In [5]:
m1.layers[17].get_weights()[0].shape

(128, 32)

In [6]:
concat_model = Model(inputs=m1.input,outputs=m1.get_layer('concatenate_2').output)

In [7]:
ppg_lstm = Model(inputs=m1.input,outputs=m1.get_layer('bidirectional_4').output)
ecg_lstm = Model(inputs=m1.input,outputs=m1.get_layer('bidirectional_5').output)

In [149]:
weights = m1.layers[15].get_weights()
h=32

# W: input weights (d, 4h)
# U: recurrent weights (h, 4h)
# b: biases (4h,)
W = weights[3]
U = weights[4]
b = weights[5]

# Extract weights for each gate
W_i = W[:, :h]
W_f = W[:, h:2*h]
W_c = W[:, 2*h:3*h]
W_o = W[:, 3*h:]

U_i = U[:, :h]
U_f = U[:, h:2*h]
U_c = U[:, 2*h:3*h]
U_o = U[:, 3*h:]

b_i = b[:h]
b_f = b[h:2*h]
b_c = b[2*h:3*h]
b_o = b[3*h:]

In [150]:
b_i.shape

(32,)

In [151]:
fwi = W_i.flatten()
fwf = W_f.flatten()
fwc = W_c.flatten()
fwo = W_o.flatten()
fui = U_i.flatten()
fuf = U_f.flatten()
fuc = U_c.flatten()
fuo = U_o.flatten()

In [148]:
np.save('lstm_forward_wts_ecg.npy',np.array([fwi,fwf,fwc,fwo,fui,fuf,fuc,fuo,b_i,b_f,b_c,b_o],dtype=object))

In [152]:
np.save('lstm_backward_wts_ecg.npy',np.array([fwi,fwf,fwc,fwo,fui,fuf,fuc,fuo,b_i,b_f,b_c,b_o],dtype=object))

In [13]:
for i in np.load('lstm_backward_wts.npy',allow_pickle=True)[0]:
    print(i)

-0.038430784
0.11252608
0.05863419
0.081588134
0.013978583
0.095248766
-0.122458085
0.12965646
0.15255676
0.0059916414
0.19722521
0.15758564
-0.12011104
0.12478454
-0.14635141
0.11703439
-0.05394246
-0.025747685
-0.17705272
0.027535874
-0.10849003
-0.08728401
0.07306128
0.14845663
-0.049260784
-0.012144314
0.030082308
-0.034021184
-0.096039474
0.14321238
-0.056200787
-0.004748111
-0.030774962
0.07266639
-0.044402182
0.10960603
-0.06226805
-0.092113785
0.104421444
-0.007226386
0.22742721
0.19077484
-0.1336624
-0.09628305
0.12476969
0.13148552
0.21672462
0.024898326
0.16178092
0.12344775
0.20126595
-0.042891804
0.072456636
-0.14601535
-0.025541876
0.008552568
-0.047404043
0.06317982
-0.13002299
0.059982836
0.17336033
-0.12944925
0.21322149
-0.15325749
-0.019670842
0.073577866
0.11379794
0.06999319
0.13301788
-0.07481334
-0.05469815
0.14937818
-0.10177388
0.123253316
-0.0008120933
0.11324449
0.031114936
0.114949
0.14300393
-0.027542267
0.1567699
0.13424218
0.11810281
0.13226318
-0.0629580

In [14]:
print(end="{")
for i in range(32):
    # print("{",end="")
    print(f"{b_o[i]}",end="")
    # if(j!=31): print(end=",")
    # print("}",end="")
    if(i!=31): print(end=",")
print(end="};")

{0.09848511219024658,0.08747454732656479,0.08485840260982513,0.10057887434959412,0.08447055518627167,0.08369029313325882,0.09496475756168365,0.08991868793964386,0.0817987471818924,0.07476145774126053,0.0626029521226883,0.09912056475877762,0.09768567979335785,0.09284835308790207,0.07483921945095062,0.06590209901332855,0.09136856347322464,0.07960332930088043,0.08710546046495438,0.08558017015457153,0.0882045179605484,0.0818815752863884,0.084994375705719,0.07780250161886215,0.09981203824281693,0.07826592773199081,0.07862535864114761,0.08966156840324402,0.06853779405355453,0.07442577928304672,0.06403539329767227,0.08866844326257706};

In [15]:
print("{")
for i in range(32):
    # print("{",end="")
    for j in range(32):
        print(f"{W_i[i][j]}",end="")
        if(i!=31 or j!=31):
            print(end=",")
    print()
print(end="};")
    # if(i!=31): print(end=",")

{
-0.03843078389763832,0.11252608150243759,0.0586341917514801,0.08158813416957855,0.013978582806885242,0.0952487662434578,-0.1224580854177475,0.12965646386146545,0.15255676209926605,0.00599164143204689,0.19722521305084229,0.15758563578128815,-0.12011104077100754,0.12478453665971756,-0.1463514119386673,0.11703439056873322,-0.05394246056675911,-0.02574768476188183,-0.17705272138118744,0.027535874396562576,-0.10849002748727798,-0.08728401362895966,0.07306127995252609,0.1484566330909729,-0.049260783940553665,-0.012144314125180244,0.030082307755947113,-0.03402118384838104,-0.09603947401046753,0.14321237802505493,-0.0562007874250412,-0.004748111125081778,
-0.030774962157011032,0.07266639173030853,-0.04440218210220337,0.10960602760314941,-0.06226804852485657,-0.09211378544569016,0.10442144423723221,-0.007226386107504368,0.22742721438407898,0.1907748430967331,-0.1336624026298523,-0.09628304839134216,0.12476968765258789,0.13148552179336548,0.21672461926937103,0.02489832602441311,0.1617809236049

In [16]:
bilstm_ppg = m1.layers[14].get_weights()
for i in bilstm_ppg:
    print(i.shape)

(32, 128)
(32, 128)
(128,)
(32, 128)
(32, 128)
(128,)


In [17]:
bilstm_wt = [bilstm_ppg[0],bilstm_ppg[1],bilstm_ppg[2],bilstm_ppg[3],bilstm_ppg[4],bilstm_ppg[5]]
bilstm_wt[0].shape

(32, 128)

In [18]:
for i in bilstm_ppg[4]:
    print("{",end="")
    for j in i:
        print(f"{j}",end=",")
    print("},")

{-0.007800644263625145,0.018908370286226273,-0.048790961503982544,0.010686245746910572,0.027697229757905006,-0.01874140277504921,-0.07064554840326309,-0.03785451874136925,-0.04162830486893654,0.021328069269657135,-0.048176784068346024,-0.049782563000917435,0.18597732484340668,0.06290213018655777,-0.08089444041252136,-0.0034865105990320444,-0.10672152042388916,-0.10213232040405273,-0.0013950066640973091,-0.0049042715691030025,0.019574005156755447,-0.01507707592099905,-0.0942535474896431,0.05545864999294281,-0.1607780009508133,0.18505804240703583,0.008655493147671223,-0.012821109034121037,-0.12695802748203278,0.10759897530078888,-0.18031659722328186,-0.0754338726401329,-0.12722566723823547,-0.04549752175807953,-0.1150704026222229,-0.16599969565868378,0.030013348907232285,-0.19325245916843414,0.09951252490282059,-0.11456533521413803,0.002602282678708434,-0.1988426148891449,0.009009651839733124,-0.024606389924883842,-0.20893020927906036,-0.14718011021614075,-0.16496197879314423,-0.04939790

In [19]:
for i in bilstm_ppg[5]:
    print(f"{i}",end=",")

0.056539297103881836,0.036406680941581726,0.04053986817598343,0.037344057112932205,0.0463993065059185,0.03805074840784073,0.0468723364174366,0.040732577443122864,0.04256743937730789,0.03566157817840576,0.024975167587399483,0.049315281212329865,0.050427839159965515,0.038739871233701706,0.024108488112688065,0.03503796085715294,0.04182155430316925,0.030684808269143105,0.03352201730012894,0.03820960223674774,0.045573629438877106,0.046968426555395126,0.037724874913692474,0.034171294420957565,0.048734746873378754,0.012730561196804047,0.02922937460243702,0.03716253861784935,0.019175564870238304,0.0317416787147522,0.015821093693375587,0.03768596053123474,1.0581642389297485,1.0345865488052368,1.0416617393493652,1.0417380332946777,1.052065372467041,1.0396016836166382,1.0488014221191406,1.0422455072402954,1.0437710285186768,1.0354477167129517,1.0276480913162231,1.051792860031128,1.0517826080322266,1.039361834526062,1.0292930603027344,1.0369471311569214,1.0426952838897705,1.0308786630630493,1.0351

In [20]:
ppg_16_ker = m1.layers[2].get_weights()[0]
ppg_16_ker.shape

(3, 1, 16)

In [21]:
# idx=0
kernel_ppg_16 = np.array([],dtype=float)
for i in range(16):
    for j in range(3):
        # print(f"{(ppg_16_ker[j][0][i])}",end=",")
        kernel_ecg_16 = np.append(kernel_ppg_16,float(f"{(ppg_16_ker[j][0][i])}"))
        # print("m")
        #  if(i!=16 or j!=3): print(end=",")
print(kernel_ppg_16.shape)
for i in kernel_ppg_16:
    print(f"{i}",end=",")

(48,)
0.06423234939575195,0.22437041997909546,0.06833374500274658,0.03577197715640068,-0.26222890615463257,0.33370164036750793,0.09259214252233505,0.0328238382935524,0.38776999711990356,0.3599998652935028,0.16379667818546295,-0.3010917603969574,0.38130825757980347,0.3348546028137207,0.2776713967323303,-0.050924140959978104,-0.34922969341278076,0.02266465499997139,0.11752639710903168,0.15204942226409912,0.37788233160972595,0.13071809709072113,0.298990935087204,0.29410573840141296,-0.4292188882827759,-0.4327573776245117,-0.026952242478728294,-0.15782462060451508,-0.3347473740577698,-0.25301599502563477,-0.036140698939561844,0.3245862126350403,-0.072092704474926,0.17751818895339966,-0.16681918501853943,0.13687556982040405,0.04302390292286873,-0.024403398856520653,0.2655993402004242,0.03788968175649643,0.27407804131507874,0.18707948923110962,-0.06529833376407623,-0.3928520381450653,-0.22526977956295013,-0.0509391687810421,-0.1765354871749878,0.3766654431819916,

In [22]:
ppg_16_bias = m1.layers[2].get_weights()[1]
for i in ppg_16_bias:
    print(f"{i}",end=",")
ppg_16_bias.shape

0.1068195328116417,0.11448175460100174,0.015388328582048416,0.12751586735248566,0.0938473641872406,0.1034792885184288,0.10392505675554276,0.0281108096241951,0.09672196209430695,0.12664088606834412,-0.10665737837553024,-0.09372849017381668,0.11298663914203644,0.09134439378976822,0.11491400003433228,0.11101774126291275,

(16,)

In [23]:
ecg_16_ker = m1.layers[3].get_weights()[0]
ecg_16_ker

array([[[-0.0913954 ,  0.18624896, -0.14387807,  0.04641822,
          0.15022212,  0.32838947,  0.19782715,  0.19269587,
         -0.35184404,  0.36981824, -0.1906616 ,  0.3293479 ,
         -0.39171475, -0.1354238 , -0.34182245, -0.09543948]],

       [[-0.02400085, -0.39798024,  0.36858124,  0.24326414,
          0.26794162,  0.16962932,  0.32896096, -0.32142147,
         -0.03733089,  0.04366583,  0.34250382, -0.3225432 ,
         -0.15851626, -0.14158435, -0.18506424, -0.38275608]],

       [[ 0.15264244, -0.31184056, -0.09850852,  0.16245358,
          0.19436252, -0.0037718 ,  0.17821284, -0.35251287,
         -0.14617723,  0.2737098 ,  0.28269163,  0.0866859 ,
         -0.0539621 , -0.3434718 , -0.16951309, -0.4265196 ]]],
      dtype=float32)

In [153]:
kernel_ecg_16 = np.array([],dtype=float)
for i in range(16):
    for j in range(3):
        # print(f"{(ppg_16_ker[j][0][i])}",end=",")
        kernel_ecg_16 = np.append(kernel_ecg_16,float(f"{(ecg_16_ker[j][0][i])}"))
        # print("m")
        #  if(i!=16 or j!=3): print(end=",")
print(kernel_ecg_16.shape)
for i in kernel_ecg_16:
    print(f"{i}",end=",")

(48,)
-0.09139540046453476,-0.024000847712159157,0.15264244377613068,0.18624895811080933,-0.3979802429676056,-0.31184056401252747,-0.14387807250022888,0.36858123540878296,-0.09850852191448212,0.04641822353005409,0.24326413869857788,0.1624535769224167,0.15022212266921997,0.2679416239261627,0.19436252117156982,0.3283894658088684,0.1696293205022812,-0.0037718035746365786,0.19782714545726776,0.32896095514297485,0.17821283638477325,0.19269587099552155,-0.32142147421836853,-0.3525128662586212,-0.35184404253959656,-0.03733088821172714,-0.1461772322654724,0.36981824040412903,0.04366583377122879,0.27370980381965637,-0.19066159427165985,0.3425038158893585,0.28269162774086,0.3293479084968567,-0.322543203830719,0.0866859033703804,-0.39171475172042847,-0.15851625800132751,-0.0539621040225029,-0.13542379438877106,-0.1415843516588211,-0.34347179532051086,-0.34182244539260864,-0.18506424129009247,-0.16951309144496918,-0.09543947875499725,-0.3827560842037201,-0.42651960253715515,

In [24]:
kernel_ecg_16 = [] 
for i in range(16):
    for j in range(3):
        kernel_ecg_16.append(ecg_16_ker[j][0][i])
print(kernel_ecg_16)

[-0.0913954, -0.024000848, 0.15264244, 0.18624896, -0.39798024, -0.31184056, -0.14387807, 0.36858124, -0.09850852, 0.046418224, 0.24326414, 0.16245358, 0.15022212, 0.26794162, 0.19436252, 0.32838947, 0.16962932, -0.0037718036, 0.19782715, 0.32896096, 0.17821284, 0.19269587, -0.32142147, -0.35251287, -0.35184404, -0.03733089, -0.14617723, 0.36981824, 0.043665834, 0.2737098, -0.1906616, 0.34250382, 0.28269163, 0.3293479, -0.3225432, 0.0866859, -0.39171475, -0.15851626, -0.053962104, -0.1354238, -0.14158435, -0.3434718, -0.34182245, -0.18506424, -0.16951309, -0.09543948, -0.38275608, -0.4265196]


In [25]:
m1.weights[0]

<KerasVariable shape=(3, 1, 16), dtype=float32, path=conv1d_8/kernel>

In [26]:
ppg_32_ker = m1.layers[8].get_weights()[0]
print(ppg_32_ker)

[[[-0.04626741  0.14097194 -0.10766277 ...  0.04744002 -0.23844312
    0.12927581]
  [-0.07101625  0.14391245  0.12320584 ...  0.01488375 -0.13550064
    0.1635188 ]
  [-0.15087648 -0.14948243 -0.05274995 ... -0.01921948  0.08878144
   -0.06026052]
  ...
  [ 0.20263243 -0.00730935  0.14178026 ... -0.09851236 -0.2594997
   -0.00943039]
  [ 0.03854539  0.12997563  0.25320536 ...  0.30700627 -0.30465177
   -0.04725601]
  [ 0.08105053 -0.10358732  0.08893935 ...  0.07807376  0.11757702
    0.0042358 ]]

 [[-0.04100424 -0.09579071  0.15342028 ...  0.19958311 -0.091125
    0.13975865]
  [ 0.2522312  -0.0961948   0.06183016 ...  0.24524643 -0.27237436
    0.2504777 ]
  [-0.29588637  0.02379427 -0.11232536 ...  0.15058465  0.02387579
   -0.29610452]
  ...
  [-0.14721093  0.05301223  0.12432573 ... -0.01359606 -0.03421271
   -0.13374889]
  [ 0.172884    0.21976759  0.22175907 ...  0.10592961 -0.08561226
    0.18979402]
  [ 0.05052474 -0.13950126  0.27090853 ...  0.08594142 -0.12221351
   -0.081

In [27]:
kernel_ppg_32 = np.array([],dtype=float)
for k in range(32):
    for i in range(16):
        for j in range(3):
            kernel_ppg_32 = np.append(kernel_ppg_32,float(f"{ppg_32_ker[j][i][k]}"))
for i in kernel_ppg_32:
    print(f"{i}",end=",")

-0.04626741260290146,-0.0410042367875576,0.05047592520713806,-0.07101625204086304,0.2522312104701996,-0.10099125653505325,-0.1508764773607254,-0.295886367559433,-0.29525113105773926,-0.044127922505140305,0.03193948417901993,0.15538810193538666,0.0835258737206459,-0.17057141661643982,-0.010573583655059338,-0.04203242436051369,0.2834734618663788,0.28992366790771484,0.19843918085098267,-0.10065275430679321,-0.025087149813771248,0.08888784795999527,-0.22914554178714752,-0.2967884838581085,0.23461763560771942,0.13320626318454742,0.05565718188881874,-0.007477898616343737,-0.059789612889289856,0.12344659119844437,-0.25934362411499023,-0.29051584005355835,-0.23333533108234406,0.04205255210399628,0.07095716148614883,-0.09618370980024338,0.14822816848754883,0.17912118136882782,-0.013038485310971737,0.20263242721557617,-0.14721092581748962,-0.1889679878950119,0.0385453924536705,0.172884002327919,0.17022936046123505,0.08105053007602692,0.050524741411209106,0.03616313263773918,0.14097194373607635,-

In [28]:
ppg_32_bias =  m1.layers[8].get_weights()[1]
for i in ppg_32_bias:
    print(f"{i}",end=",")
ppg_32_bias.shape

0.06836526840925217,0.07318256795406342,0.06358355283737183,-0.05799906700849533,-0.0902465283870697,0.044084254652261734,-0.03030713088810444,0.10529988259077072,-0.03938010334968567,0.07417023181915283,0.05863947048783302,-0.053961798548698425,0.07050304859876633,0.07546491175889969,0.0631408840417862,-0.08130943775177002,0.10333026945590973,0.08417106419801712,0.0999922826886177,0.09932158887386322,0.08650531619787216,0.06203988939523697,0.026722541078925133,0.08564529567956924,0.06743791699409485,-0.08416924625635147,-0.0918317511677742,0.10588228702545166,0.0712854415178299,0.10929620265960693,-0.08481524139642715,0.08458809554576874,

(32,)

In [154]:
ecg_32_ker = m1.layers[9].get_weights()[0]
print(ecg_32_ker)

[[[-0.11031177 -0.04910589 -0.18292123 ...  0.02517227  0.15166265
    0.22181962]
  [ 0.0972032   0.00228608  0.2389697  ...  0.12281279  0.03290657
   -0.00872072]
  [-0.10907741 -0.12836908 -0.12476519 ... -0.05164009  0.24447073
    0.13933064]
  ...
  [ 0.29912928  0.10883117  0.28360307 ...  0.00375063  0.07750401
    0.03695005]
  [-0.07055999  0.00390815 -0.09685014 ...  0.02113072  0.02190245
    0.05082698]
  [ 0.15172549  0.088271    0.12192021 ...  0.03291456  0.09129439
   -0.04501688]]

 [[ 0.18982923 -0.02453911 -0.13051876 ...  0.11659731  0.17701612
   -0.08495656]
  [ 0.2669431   0.0551704   0.06099468 ...  0.23317127 -0.0778604
    0.14067803]
  [ 0.01737888  0.03001836 -0.21782991 ... -0.08944025  0.1850502
    0.09190922]
  ...
  [ 0.09917113  0.26508024  0.09231029 ...  0.10089266  0.05401001
    0.3189774 ]
  [-0.02130209  0.05590575  0.36230955 ...  0.14754857 -0.03215387
    0.22032824]
  [ 0.18645804  0.17629956  0.21581973 ...  0.2523566  -0.0841115
    0.034

In [155]:
kernel_ecg_32 = np.array([],dtype=float)
for k in range(32):
    for i in range(16):
        for j in range(3):
            kernel_ecg_32 = np.append(kernel_ecg_32,float(f"{ecg_32_ker[j][i][k]}"))
for i in kernel_ecg_32:
    print(f"{i}",end=",")

-0.11031176894903183,0.18982923030853271,-0.08142300695180893,0.09720319509506226,0.266943097114563,0.13173678517341614,-0.10907740890979767,0.017378875985741615,-0.16961316764354706,-0.18488265573978424,0.15723039209842682,-0.039528828114271164,-0.0762733742594719,0.12615545094013214,-0.09217088669538498,0.024634361267089844,0.038041725754737854,0.1619310826063156,0.03125150501728058,0.15485386550426483,0.18513372540473938,-0.023949239403009415,0.3043801784515381,0.07861629128456116,0.2492055743932724,0.3524337112903595,0.2507933974266052,-0.18748122453689575,0.05759241431951523,-0.16222675144672394,-0.059000249952077866,-0.0018444756278768182,-0.10286853462457657,-0.10218527913093567,0.06452415883541107,-0.02090345323085785,0.18119581043720245,-0.06321900337934494,0.17553798854351044,0.29912927746772766,0.09917113184928894,-0.10620947182178497,-0.07055999338626862,-0.02130209282040596,0.1722422093153,0.15172548592090607,0.1864580363035202,-0.03455930948257446,-0.04910589009523392,-0.

In [156]:
ecg_16_bias =  m1.layers[3].get_weights()[1]
ecg_32_bias =  m1.layers[9].get_weights()[1]
print(ecg_16_bias.shape,ecg_32_bias.shape)

(16,) (32,)


In [29]:
ip_layer = Input(shape=(1024,1), name='test_input')
conv1 = Conv1D(16, kernel_size=3, kernel_constraint=max_norm(3))(ip_layer)
leaky1 = LeakyReLU(negative_slope=0.3)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(leaky1)
conv2 = Conv1D(32, kernel_size=3, kernel_constraint=max_norm(3))(max_pool1)
leaky2 = LeakyReLU(negative_slope=0.3)(conv2)
max_pool2 = MaxPooling1D(pool_size=2)(leaky2)
bilstm = Bidirectional(LSTM(32,activation="tanh", return_sequences=False, kernel_constraint=max_norm(3)))(max_pool2)
test_model = Model(inputs=ip_layer,outputs=max_pool2)
bimodel = Model(inputs=ip_layer,outputs=bilstm)
test_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ test_input (InputLayer)         │ (None, 1024, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1022, 16)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1022, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 511, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 509, 32)        │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 509, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 254, 32)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,632 (6.38 KB)

 Trainable params: 1,632 (6.38 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
bimodel.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ test_input (InputLayer)         │ (None, 1024, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1022, 16)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1022, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 511, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 509, 32)        │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 509, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 254, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        16,640 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,272 (71.38 KB)

 Trainable params: 18,272 (71.38 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
max_1 = Model(inputs=ip_layer,outputs=max_pool1)
max_1.layers[1].set_weights([ppg_16_ker,ppg_16_bias])

In [32]:
test_model.layers[1].set_weights([ppg_16_ker,ppg_16_bias])
test_model.layers[4].set_weights([ppg_32_ker,ppg_32_bias])
# test_model.layers[7].set_weights(bilstm_wt)

In [33]:
bimodel.layers[1].set_weights([ppg_16_ker,ppg_16_bias])
bimodel.layers[4].set_weights([ppg_32_ker,ppg_32_bias])
bimodel.layers[7].set_weights(bilstm_wt)

In [137]:
test_input = ppg_test[1]
test_input=np.expand_dims(test_input,axis=0)
test_input.shape

(1, 1024, 1)

In [132]:
idx = 38
ppg_input = ppg_test[idx]
ppg_input=np.expand_dims(ppg_input,axis=0)
ppg_input.shape
ecg_input = ecg_test[idx]
ecg_input=np.expand_dims(ecg_input,axis=0)
ecg_input.shape

(1, 1024, 1)

In [133]:
ppgop = ppg_lstm.predict([ppg_input,ecg_input])
ecgop = ecg_lstm.predict([ecg_input,ecg_input])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [134]:
print("{")
for i in ppgop[0]:
    print(f"{i}",end=",")
print("\n};\n{")
for i in ecgop[0]:
    print(f"{i}",end=",")
print("\n};")

{
0.9965884685516357,0.9953402280807495,-0.9976345896720886,0.9980317950248718,-0.9946983456611633,0.9959136843681335,0.995067834854126,0.9957820177078247,0.9968215227127075,0.9956980347633362,0.9954906702041626,-0.9958710074424744,0.995869517326355,0.9981555938720703,0.9955520629882812,0.9974410533905029,0.9919921159744263,0.9985225200653076,0.9966452717781067,-0.9969984889030457,0.9949290752410889,-0.995833694934845,0.9956985712051392,-0.995528519153595,0.9936937689781189,-0.9981105923652649,-0.9973344802856445,-0.9943596124649048,0.9959948658943176,0.994669497013092,0.9964267015457153,0.9945480227470398,-0.9941883087158203,-0.9932186007499695,0.9976599812507629,0.9933790564537048,-0.9983279705047607,-0.9948232769966125,0.9975493550300598,0.9971052408218384,0.9978008270263672,0.9932178854942322,-0.994038999080658,-0.990826427936554,0.9980947375297546,-0.9944567680358887,0.9952878355979919,-0.9866926670074463,0.991095244884491,0.9923940896987915,0.9968992471694946,0.9985838532447815,-

In [135]:
bpop = m1.predict([ppg_input,ecg_input])
bpop

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


array([[119.80809,  55.29941]], dtype=float32)

In [111]:
bp = m1.predict([ppg_test,ecg_test])

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


In [116]:
bp.shape

(200, 2)

In [119]:
actual = bp_test.transpose()
actual.shape

(2, 200)

In [131]:
for i in range(200):
    print(f"{i}    {bp_test[i]}   {bp[i]}")

0    [148.6311269   60.43028711]   [122.405396  56.453312]
1    [120.8605742   52.60759622]   [122.81581   56.640057]
2    [141.1594613   70.09128963]   [136.26299   65.830505]
3    [151.5632556   70.38435426]   [133.60313  64.50865]
4    [123.0118142  56.3233744]   [124.82026   57.650723]
5    [125.3586215   54.36770167]   [121.494644  55.88364 ]
6    [120.2738724   53.78099986]   [121.2661   55.76699]
7    [133.5724469   54.95440349]   [122.88206   56.719845]
8    [116.0535918   64.66959406]   [135.82372  65.76285]
9    [120.665007   52.8031635]   [123.36948   56.935818]
10    [115.9713924   51.43419259]   [120.27137   55.413525]
11    [129.2699669   55.34553803]   [123.600006  57.015087]
12    [111.4733452   51.43419259]   [124.51677  57.42673]
13    [123.598516   54.1721344]   [124.72223   57.517323]
14    [119.8827379   54.56326895]   [124.9549    57.597958]
15    [132.2209903   67.35601979]   [137.36125  66.56497]
16    [119.1004688   53.97656713]   [123.1646    56.815495]
17    

In [121]:
sbp_mae = mean_absolute_error(bp.transpose()[0],actual[0])
dbp_mae = mean_absolute_error(bp.transpose()[1],actual[1])
print(sbp_mae,dbp_mae)

7.3719831695366205 3.4171702752563973


In [42]:
ppg_test[0]

array([[-1.00654436],
       [-0.65374577],
       [-0.30088938],
       ...,
       [-0.8716711 ],
       [-0.90638354],
       [-0.93122876]])

In [43]:
for i in range(1024):
    print(f"{ppg_test[0].transpose()[0][i]}",end="")
    if(i!=1023):
        print(end=",")

-1.0065443567064287,-0.6537457696640265,-0.30088938370391677,0.040571408860962534,0.3660972329389812,0.6683319507531369,0.9339924394494553,1.14917294688093,1.3068364480654133,1.4090466266389017,1.46360713152906,1.4797695786686538,1.4658638271097086,1.4286334196131532,1.3732169336170736,1.3032198176506808,1.2208539160189835,1.1272919940147617,1.023281814736616,0.9098654685349243,0.7889314147189836,0.663392719052164,0.5369717940435387,0.4137100347694497,0.29734215316735146,0.1906700414430455,0.09510919774299158,0.010592351180104173,-0.06409377063502328,-0.1306593259303081,-0.1906216700310876,-0.24485545712133255,-0.29353441708306516,-0.33642670620343323,-0.3733477799079976,-0.4045244365188556,-0.4307076054572639,-0.45303546654177923,-0.47277994879136365,-0.49112252510062354,-0.5090230043688293,-0.5271689342262502,-0.5459833540771893,-0.565685491807634,-0.5863821113475212,-0.6081332141529703,-0.6309464379077713,-0.6547187075888147,-0.6791950790724258,-0.7039982084301207,-0.728720795549562

In [44]:
max_op1 = max_1.predict(test_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [45]:
max_op1.shape

(1, 511, 16)

In [46]:
for i in max_op1[0][0]:
    print(f"{i}")

8.963048458099365e-05
0.1835366189479828
-0.011786208488047123
-0.05079987272620201
-0.07347600162029266
0.3762245774269104
-0.0009978488087654114
-0.04061295464634895
0.8197727203369141
0.5804678797721863
-0.055086005479097366
-0.046209871768951416
0.1029784083366394
-0.0024908760096877813
0.5052462816238403
0.21271851658821106


In [47]:
l1op = max_op1[0].transpose()
l1op.shape

(16, 511)

In [48]:
print(end="{")
for i in range(16):
    for j in range(511):
        print(f"{l1op[i][j]}",end="")
        if(i!=15 or j!=510):
            print(end=",")
print(end="};")

{8.963048458099365e-05,0.23723655939102173,0.4378356337547302,0.5701344609260559,0.6268342137336731,0.6330149173736572,0.6153555512428284,0.5708539485931396,0.508093535900116,0.43060487508773804,0.34303343296051025,0.25445330142974854,0.17519840598106384,0.11092546582221985,0.060360658913850784,0.019578829407691956,-0.0039093978703022,-0.011206995695829391,-0.016440125182271004,-0.020557517185807228,-0.024439925327897072,-0.028572915121912956,-0.03312215954065323,-0.03810573369264603,-0.04336780682206154,-0.04860784485936165,-0.053640276193618774,-0.05857880413532257,-0.0636540874838829,-0.06878234446048737,-0.07354971766471863,-0.07770846039056778,-0.08174201846122742,-0.08686090260744095,-0.09359905123710632,-0.0998523086309433,-0.09685923159122467,-0.07739780098199844,-0.039215490221977234,0.051024168729782104,0.25679662823677063,0.4462660551071167,0.5889137983322144,0.6747620701789856,0.7116847038269043,0.7164700627326965,0.7065766453742981,0.6758230328559875,0.6266965866088867,0.5

In [49]:
for i in max_op1[0].transpose()[0]:
    print(f"{i}",end=", ")

8.963048458099365e-05, 0.23723655939102173, 0.4378356337547302, 0.5701344609260559, 0.6268342137336731, 0.6330149173736572, 0.6153555512428284, 0.5708539485931396, 0.508093535900116, 0.43060487508773804, 0.34303343296051025, 0.25445330142974854, 0.17519840598106384, 0.11092546582221985, 0.060360658913850784, 0.019578829407691956, -0.0039093978703022, -0.011206995695829391, -0.016440125182271004, -0.020557517185807228, -0.024439925327897072, -0.028572915121912956, -0.03312215954065323, -0.03810573369264603, -0.04336780682206154, -0.04860784485936165, -0.053640276193618774, -0.05857880413532257, -0.0636540874838829, -0.06878234446048737, -0.07354971766471863, -0.07770846039056778, -0.08174201846122742, -0.08686090260744095, -0.09359905123710632, -0.0998523086309433, -0.09685923159122467, -0.07739780098199844, -0.039215490221977234, 0.051024168729782104, 0.25679662823677063, 0.4462660551071167, 0.5889137983322144, 0.6747620701789856, 0.7116847038269043, 0.7164700627326965, 0.7065766453742

In [50]:
maxop = test_model.predict(test_input)
maxop.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


(1, 254, 32)

In [51]:
lstmip = maxop[0].flatten()
# print(end="{")
# for i in range(254):
#     for j in range(32):
#         print(f"{lstmip[i*32+j]}",end="")
#         if(i!=31 or j!=253):
#             print(end=",")
#     print()
# print(end="};")

In [52]:
lstmip.shape

(8128,)

In [53]:
np.save('lstmip.npy',lstmip)

In [54]:
print(end="{")
for i in range(254):
    for j in range(32):
        print(f"{maxop[0][i][j]}",end="")
        if(i!=31 or j!=253):
            print(end=",")
    print()
print(end="};")

{-0.12783820927143097,0.8422307372093201,1.5352816581726074,-0.1774718314409256,-0.2666282057762146,0.8785587549209595,0.025465836748480797,0.2845938801765442,-0.2817780077457428,0.0402042418718338,0.8172240257263184,-0.295515239238739,-0.22556830942630768,1.4744819402694702,1.2298610210418701,-0.3049475848674774,1.3270282745361328,0.8755148649215698,1.2704447507858276,0.947109580039978,1.6017206907272339,0.821306049823761,1.1420698165893555,-0.002681004349142313,1.4108885526657104,-0.31905385851860046,-0.3950707018375397,-0.08984073996543884,1.1604927778244019,1.0755174160003662,-0.28024518489837646,-0.11760915070772171,
-0.3923322260379791,0.8906965255737305,1.7767424583435059,-0.15163524448871613,-0.18231883645057678,1.186870813369751,-0.09311210364103317,-0.3396860957145691,-0.44440504908561707,-0.342764675617218,0.9462857246398926,-0.33800482749938965,-0.6270961165428162,1.749735951423645,1.440314769744873,-0.30038174986839294,1.5307170152664185,1.1568411588668823,1.74530589580535

In [139]:
lstmop = bimodel.predict(test_input)
lstmop.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step


(1, 64)

In [144]:
for i in lstmop[0][32:]:
    print(f"{i}")

-0.994171142578125
-0.9909980297088623
0.9964936971664429
0.9946187734603882
-0.9974949955940247
-0.9869460463523865
0.9892994165420532
0.9955329895019531
0.9950671792030334
0.9955993294715881
-0.9942177534103394
-0.9973068833351135
0.9957067370414734
-0.9901389479637146
0.9949849843978882
-0.9845466613769531
0.9917710423469543
0.994336724281311
0.9935926795005798
0.9977302551269531
-0.9933850169181824
-0.9966561198234558
0.9911578893661499
-0.9933810830116272
-0.9933436512947083
0.988322377204895
-0.9936414957046509
-0.9944162368774414
-0.99162358045578
0.9957364201545715
0.9976342916488647
-0.9911912083625793


In [142]:
test_input.shape

(1, 1024, 1)

In [143]:
for i in test_input[0]:
    print(f"{i[0]}",end=",")

-0.8247519896168687,-0.5060038029423554,-0.17645225656784144,0.15059679381260727,0.47077715605717146,0.778821849665477,1.0636513465657624,1.3113620046201002,1.5115304615528173,1.660135709027666,1.75804182703383,1.8086212573039577,1.816736333057299,1.7885426847096029,1.7309876665405792,1.6507638029119096,1.5532714019349554,1.4422142175637216,1.3200037157604667,1.1886461045681385,1.0505722318909663,0.9090253121179412,0.7679606242069899,0.6316437685480227,0.5041376734018228,0.38877087373957386,0.28764795069278465,0.2013116382123597,0.12869630897971304,0.06745311037955032,0.014591452113390165,-0.03275865460926081,-0.07672203605408297,-0.11839137689186809,-0.1579244167696562,-0.19491081129976742,-0.22881613761806757,-0.2593430476097126,-0.2866373252210739,-0.3113290037046722,-0.3344159063877363,-0.35702172804275084,-0.3801185517225977,-0.40433669371900893,-0.42993425460321893,-0.4568981315165495,-0.4850828644034956,-0.5143030286591168,-0.5443482443336868,-0.5749383942804894,-0.6056655327749

In [59]:
print(f"{lstmop[0][1][0]}")

IndexError: invalid index to scalar variable.

In [ ]:
print("{")
for i in range(254):
    for j in range(32):
        print(f"{maxop[0][i][j]}",end="")
        if(i!=253 or j!=31): print(end=",")
    print()
print("};")

{
-0.12783820927143097,0.8422307372093201,1.5352816581726074,-0.1774718314409256,-0.2666282057762146,0.8785587549209595,0.025465836748480797,0.2845938801765442,-0.2817780077457428,0.0402042418718338,0.8172240257263184,-0.295515239238739,-0.22556830942630768,1.4744819402694702,1.2298610210418701,-0.3049475848674774,1.3270282745361328,0.8755148649215698,1.2704447507858276,0.947109580039978,1.6017206907272339,0.821306049823761,1.1420698165893555,-0.002681004349142313,1.4108885526657104,-0.31905385851860046,-0.3950707018375397,-0.08984073996543884,1.1604927778244019,1.0755174160003662,-0.28024518489837646,-0.11760915070772171,
-0.3923322260379791,0.8906965255737305,1.7767424583435059,-0.15163524448871613,-0.18231883645057678,1.186870813369751,-0.09311210364103317,-0.3396860957145691,-0.44440504908561707,-0.342764675617218,0.9462857246398926,-0.33800482749938965,-0.6270961165428162,1.749735951423645,1.440314769744873,-0.30038174986839294,1.5307170152664185,1.1568411588668823,1.7453058958053

In [ ]:
type(kernel_ppg_16)

numpy.ndarray

In [157]:
conv_wts = np.array([kernel_ecg_16,ecg_16_bias,kernel_ecg_32,ecg_32_bias],dtype=object)

In [158]:
np.save('conv_ecg_wts.npy',conv_wts)

In [ ]:
wts = np.load('conv_ppg_wts.npy',allow_pickle=True)

In [ ]:
wts.shape

(4,)

In [ ]:
wts[1].shape

(16,)

In [ ]:
ppg_test[0]

array([[-1.00654436],
       [-0.65374577],
       [-0.30088938],
       ...,
       [-0.8716711 ],
       [-0.90638354],
       [-0.93122876]])

In [ ]:
ppg_ip = np.squeeze(ppg_test)
ecg_ip = np.squeeze(ecg_test)
bp_op = np.squeeze(bp_test)
bp_op.shape

(200, 2)

In [ ]:
np.save("ppg_test.npy",ppg_ip)

In [ ]:
np.save("ecg_test.npy",ecg_ip)

In [ ]:
np.save("bp_test.npy",bp_op)

In [ ]:
ans = np.load('ans.npy')

In [ ]:
ans.shape

(8200,)

In [ ]:
for i in range(1024):
    print(ans[i],end=",")

0.4032469,0.22208554,-0.055033743,-0.057825547,-0.071054906,-0.06394346,-0.06640187,-0.09980374,-0.12899296,-0.11512752,0.19497678,0.43173948,0.38142142,0.39590985,0.39439812,0.37299284,0.3744808,0.38776782,0.3758328,0.3766816,0.40484747,0.40370655,0.42222863,0.4535917,0.5061423,0.4951762,0.410208,0.3648737,0.35248432,0.3616772,0.47048497,0.4959961,0.48426363,0.5255887,0.5312109,0.5293802,0.5770995,0.5747229,0.6118482,0.6735438,0.7718625,0.7541333,0.586722,0.4766764,0.45205823,0.4568965,0.1361764,-0.07691936,-0.10192264,-0.07338505,-0.06663647,-0.070224345,-0.07308523,-0.06998488,-0.07241295,-0.07601479,-0.08207505,-0.07821826,-0.08981908,-0.0899104,-0.09149216,-0.09677842,-0.07849948,-0.07228384,-0.09023924,-0.039894927,-0.044332623,-0.046222545,-0.0556164,-0.047298163,-0.05622824,-0.0701509,-0.096744224,-0.06917304,-0.07843274,-0.064813934,-0.06931131,-0.07866588,-0.06388668,-0.058055405,-0.071583495,-0.06592258,-0.06545302,-0.06904175,-0.082101256,-0.06999398,-0.09135492,-0.10520101

In [ ]:

# model = tf.keras.models.load_model('/content/model.keras')

# For demonstration, let's assume 'model' is your loaded model
# replace this with your actual model loading code

# Function to extract and display weights
def extract_dense_layer_weights(model):
    dense_layers = [layer for layer in model.layers if isinstance(layer, tf.keras.layers.Dense)]
    
    for i, layer in enumerate(dense_layers):
        weights, biases = layer.get_weights()
        print(f"Layer {i + 1}: {layer.name}")
        print(f"Weights:\n{weights}\n")
        print(f"Biases:\n{biases}\n")

# Extract and display dense layer weights
extract_dense_layer_weights(m1)

Layer 1: dense_4
Weights:
[[-0.00638596  0.13242133  0.00858374 ...  0.26461375  0.11806095
  -0.03036383]
 [ 0.32406434  0.00776172 -0.08987325 ...  0.17969221  0.06348563
  -0.05455676]
 [ 0.00362195 -0.06463252 -0.12957719 ... -0.20669633 -0.20793648
   0.17797574]
 ...
 [-0.36724305 -0.04767944 -0.14258368 ... -0.06565718 -0.26302588
  -0.1339749 ]
 [-0.27812663  0.01319578  0.10449427 ... -0.0242235  -0.26677614
   0.14319898]
 [ 0.14425728  0.03969388  0.07221267 ... -0.0008006   0.2388205
  -0.16876131]]

Biases:
[ 0.13928154 -0.01730692 -0.01670673  0.14960375  0.1314691  -0.01044556
 -0.0125027   0.1598601  -0.02209305 -0.01044906 -0.01217783  0.14977881
 -0.01102979  0.13465524 -0.0085102   0.16114196  0.13828304 -0.00600546
 -0.01486331  0.14125548  0.17596574 -0.01601843  0.13175988  0.13597876
 -0.01535622 -0.01570863  0.14425501 -0.02471193 -0.01122913  0.15927681
  0.13191359 -0.01304225]

Layer 2: dense_5
Weights:
[[ 0.51381654  0.06938283]
 [-0.16237576  0.33054543]
 [